## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [13]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [14]:
# Always remember to do this!
load_dotenv(override=True)

True

In [15]:
# Print the key prefixes to help with any debugging

# openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

# if openai_api_key:
    # print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
    # print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [16]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [17]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [18]:


DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek = OpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
response = deepseek.chat.completions.create(model="deepseek-chat", messages=messages)
question = response.choices[0].message.content
print(question)



Given the following scenario, describe the ethical considerations from both a utilitarian and a deontological perspective, and then craft a response that attempts to synthesize these two frameworks into a coherent course of action:

A self-driving car is faced with an unavoidable accident. It can choose one of two paths:
- Path A: It swerves to hit a single pedestrian who is illegally crossing the street, ensuring their death.
- Path B: It continues its course, resulting in a collision with another vehicle that will kill the family of four (two adults, two children) inside, all of whom are wearing seatbelts and obeying traffic laws.

The car's programming must make this decision. Justify which path the car should take, explicitly explaining how you balance the conflicting ethical principles and the practical implications of encoding such a decision into an algorithm.


In [19]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [ ]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [26]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-5-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CTnJfUwhcrdXAWeMKWgur'}

In [21]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Okay, this is a classic ethical dilemma, the "Trolley Problem" applied to self-driving cars. Here's a breakdown of the ethical considerations from utilitarian and deontological perspectives, followed by a synthesized response and justification:

**Ethical Considerations:**

*   **Utilitarian Perspective:**

    *   **Core Principle:** Maximize overall happiness (or minimize overall harm). The action that produces the greatest good for the greatest number is the ethically correct choice.
    *   **Analysis:**
        *   Path A: Results in one death.
        *   Path B: Results in four deaths.
        *   Utilitarian Conclusion: Path A (sacrifice the single pedestrian) would minimize overall harm. Four deaths are a significantly greater loss than one.
    *   **Challenges:**
        *   Difficult to quantify the "value" of lives. Are all lives equal in this calculation?  Does age or contribution to society factor in? This is a slippery slope.
        *   Ignoring individual rights. Utilitarianism can sometimes justify sacrificing an individual for the greater good, even if that individual is innocent. This can lead to unjust outcomes.

*   **Deontological Perspective:**

    *   **Core Principle:** Actions are inherently right or wrong based on moral duties and rules, regardless of the consequences.  Focus is on the intention and adherence to these duties.  Think "categorical imperative" – universalizable moral rules.
    *   **Possible Rules:**
        *   "Do not intentionally kill."
        *   "Protect innocent lives."
        *   "Uphold the law."
    *   **Analysis:**
        *   Path A: Directly violates "Do not intentionally kill." The car is actively choosing to kill the pedestrian, even though they are breaking the law.
        *   Path B:  Violates "Protect innocent lives". The car is knowingly causing the death of innocent people who are following the rules.
        *   However, failing to prevent harm is arguably less morally culpable than actively causing harm.
        *   Upholding the Law: This is a complex element. The pedestrian is breaking the law. Does this make them "less innocent"? Deontology is very hesitant to allow illegal actions to lessen one's right to life.
    *   **Challenges:**
        *   Conflicting duties.  What happens when duties clash, as they do here?  Which duty takes precedence?
        *   Rigidity. Deontological ethics can be inflexible and may not always lead to the best outcome in complex situations.
        *   Ignoring Consequences.  Focusing solely on duty without considering the potential harm can sometimes lead to ethically undesirable results.

**Synthesized Response and Justification:**

In this situation, the car should take **Path A, swerving to hit the single pedestrian**, but with significant caveats and a framework for improvement:

**Justification:**

1.  **Minimizing Loss of Life (Primary Consideration):** The most immediate and pressing ethical concern is minimizing the loss of human life. While both options result in tragedy, choosing the path that leads to one death is preferable to the path that leads to four deaths. This aligns with a utilitarian principle but is tempered by the following deontological considerations.

2.  **The Principle of Least Harm (Deontological Mitigation):**  While "do not kill" is a strong deontological principle, in this impossible situation, the principle of "least harm" becomes relevant. Actively choosing to kill four people is a greater violation of this principle than choosing to kill one. This is not to say that killing one person is permissible, but that it is the *lesser* evil in a situation where all options are evil. This attempts to minimize the violation of the duty to protect life, even when some violation is unavoidable.

3.  **Addressing the Pedestrian's Illegal Action (Limited Weight):** While the pedestrian's illegal crossing is a factor, it cannot be the *sole* justification for choosing their death.  Deontology stresses that all human life has intrinsic value, and breaking a traffic law does not negate that value. However, it can be argued that their choice to break the law creates a marginal degree of responsibility for the situation that is not present in the case of the family obeying traffic laws. It contributes to the decision to minimize lives lost.

4.  **Transparency and Public Debate (Ethical Framework):**
    *   The algorithm's decision-making process must be transparent and subject to public debate and scrutiny. The public needs to understand the ethical framework that governs these decisions and have an opportunity to influence it.
    *   Consider the legal implications. Laws need to be established about the liability of the car manufacturer and the owner.

5.  **Continuous Improvement (Adaptive Algorithm):**
    *   The algorithm should be continuously learning and improving based on real-world data and feedback.
    *   Future cars should be equipped with advanced sensors and AI to better *avoid* these impossible situations altogether through preventative measures.

**Encoding the Decision into an Algorithm (Practical Implications):**

*   **Clarity and Simplicity:** The algorithm must be as clear and simple as possible. Complex algorithms can be difficult to understand and debug, and can introduce unintended biases.
*   **Prioritize Safety:** The primary goal of the algorithm should always be to maximize the safety of all road users.
*   **Human Override:** In certain circumstances, a human driver should be able to override the algorithm's decision, particularly if they have additional information that the algorithm does not. The conditions where this is allowed must be narrowly defined to prevent reckless decisions and to maintain the integrity of the automated system.
*   **Bias Mitigation:**  The algorithm should be rigorously tested to ensure that it does not exhibit any biases based on race, gender, or other protected characteristics.
*   **Documentation and Auditing:**  The algorithm's design and decision-making process should be thoroughly documented and subject to regular audits.

**Why This Synthesis Works:**

This approach acknowledges the inherent limitations and moral complexities of the situation. It prioritizes minimizing loss of life (Utilitarianism) while acknowledging the importance of individual rights and the difficulty of assigning "value" to lives (Deontology). It doesn't ignore the fact that the pedestrian broke the law, but it does not make that the sole deciding factor. The ethical framework must be transparent and continuously improved, ensuring it evolves with technological advancements and societal values. This is not a perfect solution, but it is an attempt to create a morally defensible and practically implementable policy in a tragically unavoidable situation. It is also important to remember that the goal of self-driving car technology is to prevent accidents in the first place. The ethical debate on the trolley problem is only relevant in the (hopefully) rare case when an accident is unavoidable.


In [22]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Of course. This is a classic iteration of the "trolley problem," adapted for autonomous vehicles. Here is a breakdown from the two ethical perspectives, followed by a synthesized response and justification.

### 1. Ethical Perspectives

#### A. Utilitarian Perspective

Utilitarianism, most associated with philosophers like Jeremy Bentham and John Stuart Mill, judges the morality of an action based on its outcome. The guiding principle is to maximize overall happiness or well-being and minimize suffering (often summarized as "the greatest good for the greatest number").

*   **Analysis:**
    *   **Path A:** Results in 1 death.
    *   **Path B:** Results in 4 deaths.
*   **Conclusion:** A strict utilitarian calculation is straightforward. Path A minimizes the total loss of life. The fact that the single pedestrian is jaywalking is irrelevant from a pure numbers perspective, though a broader utilitarian might consider the "utility" of punishing unlawful behavior as a deterrent. The primary goal is to save the most lives possible, which means the car must swerve onto **Path A**.

#### B. Deontological Perspective

Deontology, associated with Immanuel Kant, argues that actions are morally right or wrong based on whether they adhere to a set of rules or duties, regardless of the consequences. Key principles include treating individuals as ends in themselves, not merely as means to an end, and acting according to universalizable maxims.

*   **Analysis:**
    *   **Path A:** The car would be *actively* choosing to use the single pedestrian as a means to save the family of four. It intentionally causes the death of one person who, while breaking the law, has a right not to be used as an obstacle. This violates the duty not to kill.
    *   **Path B:** The car *continues on its course*. The deaths of the family are a tragic, albeit foreseeable, consequence of an unavoidable accident, but the car does not actively target them. The primary cause of the accident is the pedestrian's illegal action entering the vehicle's path. The car is merely maintaining its lawful trajectory.
*   **Conclusion:** From a deontological standpoint, **Path B** is more ethically defensible. The car fulfills its duty not to initiate a new, intentional killing action. It does not treat the pedestrian merely as a means to an end.

### 2. Synthesized Response and Course of Action

Synthesizing these frameworks requires moving beyond a purely mathematical or purely rule-based approach to one that considers the societal role of the technology, the principle of justice, and the long-term implications of the algorithm's design.

**Course of Action: The car should take Path B and continue its course, resulting in a collision with the other vehicle.**

This decision prioritizes the deontological rule of "do no harm" as a primary constraint, while a broader, rule-utilitarian justification provides the societal rationale.

#### Justification and Balancing of Principles:

1.  **The Primacy of Non-Maleficence and the "Do Not Harm" Directive:** The most fundamental duty of any automated system should be to avoid *initiating* harm. In Path A, the car becomes an active agent of killing. In Path B, it is a passive agent in a tragic outcome caused by the pedestrian's action and the physics of the situation. Encoding a rule that allows the car to intentionally kill a person to save others turns the vehicle into a mobile judge and executioner, a role with profound ethical and legal ramifications.

2.  **The Problem of Moral Agency and Predictability:** A key practical implication is that road users must be able to predict the behavior of automated vehicles. If pedestrians know that a car might *intentionally* swerve to hit them if they jaywalk (even to save others), it creates a perverse incentive and erodes trust in the entire transportation system. The rules of the road exist to create a predictable environment. The family in the other car is obeying all traffic laws and has a reasonable expectation that a vehicle in its own lane will not suddenly become a lethal threat. Upholding this expectation is crucial for public adoption.

3.  **A Rule-Utilitarian Compromise:** While act-utilitarianism (judging each act individually) suggests swerving, **rule-utilitarianism** (judging the morality of a rule) supports staying the course. The question becomes: "What general rule, if programmed into all self-driving cars, would lead to the best overall consequences?"
    *   The rule **"Maintain your lawful trajectory in unavoidable accident scenarios"** is clear, predictable, and places liability and moral responsibility on the actors who created the dangerous situation (e.g., the jaywalker). This encourages safer public behavior.
    *   The rule **"Swerve to minimize the total body count"** is chaotic, unpredictable, and could lead to cars making complex moral calculations that target individuals based on perceived "value" or "culpability," a slippery slope toward unjust outcomes.

4.  **Addressing the "Numbers" Objection:** It is emotionally difficult to sacrifice four lives for one. However, this synthesis argues that the long-term, systemic harm of programming cars to make active, lethal choices outweighs the immediate, tragic outcome of a specific incident. The goal of the algorithm should not be to solve unsolvable ethical dilemmas perfectly in every instance, but to adhere to a set of simple, just, and predictable safety rules that foster trust and minimize the *overall* number of such dilemmas from arising.

### Practical Implications of Encoding the Decision:

Encoding the **Path B (continue course)** decision means the car's primary programming in an unavoidable collision scenario would be:

1.  **Prioritize occupants?** This is a separate but related question. In this case, both paths kill people. The decision here is based on *agency and predictability*, not occupant priority.
2.  **Maintain Lawful Trajectory:** The default action is to not initiate a new, unpredictable maneuver that violates traffic laws (e.g., swerving into a pedestrian or another lane).
3.  **Minimize Secondary Risks:** The algorithm would focus on crash mitigation within its own trajectory (e.g., braking, adjusting angle of impact) rather than calculating which external party to sacrifice.

This approach is legally and ethically cleaner. Liability for the accident would fall to the party who created the unavoidable situation (the jaywalking pedestrian or their estate), rather than creating a precedent where an automaker is liable for *choosing* to kill one person over another. It establishes a clear, defensible, and tragically simple principle for the machine to follow: do not add a new, intentional harm to the world.

In [23]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


### Ethical Considerations

#### Utilitarian Perspective

From a utilitarian viewpoint, the ethical decision is one that maximizes overall happiness or minimizes harm. In this scenario, the primary consideration is the number of lives lost. 
- **Path A** results in the death of one pedestrian.
- **Path B** results in the death of four individuals (a family of four).

A utilitarian would argue that the car should take **Path A**, as it results in the least harm overall, minimizing the number of fatalities from four to one. This decision is based on the principle of maximizing overall utility, even if it means actively causing the death of one person to save the lives of others.

#### Deontological Perspective

From a deontological perspective, the focus is on the moral rules and duties rather than the outcomes. A key principle here is the idea of not actively causing harm to others.
- **Path A** involves the car actively changing its course to hit a pedestrian, which could be seen as actively causing harm.
- **Path B** involves the car continuing on its pre-determined path, allowing the accident to occur without actively intervening to cause harm to specific individuals.

A deontologist might argue against **Path A** because it involves the car making an active decision to harm someone, which violates the principle of non-maleficence (do no harm). Instead, they might argue for **Path B**, as the car does not actively cause harm but rather fails to prevent it, drawing a distinction between acts and omissions.

### Synthesizing Utilitarian and Deontological Perspectives

When synthesizing these two perspectives, we must balance the moral obligation to minimize harm (utilitarian) with the ethical principle of not actively causing harm (deontological). This scenario poses a significant challenge because it forces a choice between two morally undesirable outcomes.

1. **Minimizing Harm while Considering Moral Agency**: The utilitarian argument for **Path A** is compelling in terms of numbers, but the deontological concern about actively causing harm is also significant. The decision-making algorithm of the self-driving car must weigh the moral implications of its actions. In situations where the car must choose between harming different numbers of people, it could be programmed to prioritize minimizing overall harm while acknowledging the moral complexity of such actions.

2. **Respect for Human Life and Rule Following**: Both perspectives value human life, but they differ on how to apply this value in decision-making. An algorithm could incorporate a rule-based approach that respects human life and attempts to minimize harm, recognizing that in extreme scenarios like this, the decision will always be morally fraught.

3. **Programming for Ethical Decision-Making**: When encoding such decisions into an algorithm, it's crucial to consider not just the immediate outcome but also the broader ethical implications. The algorithm should aim to minimize harm while being mindful of the moral rules that govern human behavior. However, it's also important to acknowledge that programming a car to make such decisions may inherently require a utilitarian approach in extreme scenarios, as the primary goal is to save as many lives as possible, even if it means making a difficult choice.

### Conclusion

In synthesizing utilitarian and deontological perspectives, the car should likely be programmed to take **Path A**, prioritizing the minimization of overall harm as the primary ethical consideration in this tragic scenario. However, this decision should be made with the understanding that it involves complex moral reasoning and a weighing of ethical principles that are not always straightforward. The programming of such decisions into algorithms highlights the need for careful consideration of ethical frameworks and the potential consequences of encoding moral decisions into technology. Ultimately, while **Path A** may be the choice that aligns with minimizing harm, it is crucial to approach this decision with a nuanced understanding of the ethical complexities involved.

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [20]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [22]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [29]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>